In [318]:
import gzip
from io import BytesIO
from ish_parser import ish_parser

def read_observations(years, usaf='081810', wban='99999'):
    parser = ish_parser()
    
    for year in years:
        path = "../data/observations/{usaf}-{wban}-{year}.gz".format(year=year, usaf=usaf, wban=wban)
        with gzip.open(path) as gz:
            parser.loads(bytes.decode(gz.read()))
            
    reports = parser.get_reports()
    
    station_latitudes = [41.283, 41.293] 
    observations = pd.DataFrame.from_records(((r.datetime, 
                                               r.air_temperature.get_numeric(),
                                               (r.precipitation[0]['depth'].get_numeric() if r.precipitation else 0),
                                               r.humidity.get_numeric(),
                                               r.sea_level_pressure.get_numeric(),
                                               r.wind_speed.get_numeric(),
                                               r.wind_direction.get_numeric()) 
                                              for r in reports if r.latitude in station_latitudes and r.datetime.minute == 0),
                             columns=['timestamp', 'AT', 'precipitation', 'humidity', 'pressure', 'wind_speed', 'wind_direction'], 
                             index='timestamp')
    
    return observations

In [319]:
import json
import pandas as pd
import numpy as np

nems4_lookahead = 12

def read_nems4(years, prediction_hours=12):
    predictions=pd.DataFrame()
    for year in years:
        with open('../data/NEMS4/{}.json'.format(year)) as json_data:
            d = json.load(json_data)
            if not predictions.empty:
                predictions = predictions.append(pd.DataFrame(d['history_1h']))
            else:
                predictions = pd.DataFrame(d['history_1h'])

    predictions = predictions.set_index('time')
    predictions.index.name = 'timestamp'
    
    # shift dataset back 12 hours as its a the value is the prediction for the given timestmap 12 hours previously
    predictions.index = pd.to_datetime(predictions.index) - pd.Timedelta(hours=nems4_lookahead)
    predictions.index.tz = 'UTC'

    predictions = predictions[['temperature', 'precipitation', 
                   'relativehumidity', 'sealevelpressure', 
                   'windspeed', 'winddirection']]
    
    predictions = predictions.rename(columns={
        'windspeed': 'nems4_wind_speed',
        'winddirection': 'nems4_wind_direction', 
        'temperature': 'nems4_AT',
        'precipitation': 'nems4_precipitation',
        'relativehumidity': 'nems4_humidity',
        'sealevelpressure': 'nems4_pressure'})
    
    return predictions

In [320]:
years = range(2007, 2016)
dataset = pd.merge(read_observations(years), read_nems4(years), left_index=True, right_index=True, how='inner')
# dataset = read_observations(years)

original = dataset.copy(deep=True)
dataset.describe()

,AT,precipitation,humidity,pressure,wind_speed,wind_direction,nems4_AT,nems4_precipitation,nems4_humidity,nems4_pressure,nems4_wind_speed,nems4_wind_direction
count,76096.000000,76005.000000,76073.000000,59702.000000,76145.000000,74446.000000,76161.000000,76161.000000,76161.000000,76161.000000,76161.000000,76161.000000
mean,16.882632,0.076991,67.859135,1016.604899,4.111837,235.901056,16.380258,0.036410,70.884521,1016.181576,3.506167,210.305419
std,6.782857,1.069337,14.420932,6.886706,2.112691,107.677121,6.757949,0.279013,16.583534,7.148011,1.983231,107.191350
min,-3.500000,0.000000,8.000000,980.200000,0.000000,10.000000,-5.840000,0.000000,1.000000,982.000000,0.000000,0.000000
25%,11.700000,0.000000,59.000000,1012.900000,2.600000,160.000000,11.390000,0.000000,59.000000,1012.000000,2.050000,120.000000
50%,16.800000,0.000000,69.000000,1016.900000,4.100000,240.000000,16.570000,0.000000,72.000000,1017.000000,3.140000,212.000000
75%,22.500000,0.000000,78.000000,1020.600000,5.100000,340.000000,21.540000,0.000000,85.000000,1020.000000,4.620000,318.000000
max,35.300000,75.900000,100.000000,1041.600000,27.800000,360.000000,39.210000,16.000000,100.000000,1044.000000,22.320000,360.000000


In [321]:
from sklearn import preprocessing

pd.options.mode.chained_assignment = None
np.random.seed(1234)

def drop_duplicates(df):
    print("Number of duplicates: {}".format(len(df.index.get_duplicates())))
    return df[~df.index.duplicated(keep='first')]
    
def impute_missing(df):
    # todo test with moving average / mean or something smarter than forward fill
    print("Number of rows with nan: {}".format(np.count_nonzero(df.isnull())))
    df.fillna(method='ffill', inplace=True)
    return df
    
def first_order_difference(data, columns):
    for column in columns:
        data[column+'_d'] = data[column].diff(periods=1)
    
    return data.dropna()

# def derive_prediction_columns(data, column, column2, horizons):
#     for look_ahead in horizons:
#         data['prediction_' + str(look_ahead)] = data[column].diff(periods=look_ahead).shift(-look_ahead)
        
#     for look_ahead in horizons:
#         data['prediction_direction_' + str(look_ahead)] = data[column2].diff(periods=look_ahead).shift(-look_ahead)

def derive_prediction_columns(data, column, horizons):
    for look_ahead in horizons:
        data['prediction_' + str(look_ahead)] = data[column].diff(periods=look_ahead).shift(-look_ahead)
    
    return data.dropna()

def scale_features(scaler, features):
    scaler.fit(features)
    
    scaled = scaler.transform(features)
    scaled = pd.DataFrame(scaled, columns=features.columns)
    
    return scaled

def inverse_prediction_scale(scaler, predictions, original_columns, column):
    loc = original_columns.get_loc(column)
    
    inverted = np.zeros((len(predictions), len(original_columns)))
    inverted[:,loc] = np.reshape(predictions, (predictions.shape[0],))
    
    inverted = scaler.inverse_transform(inverted)[:,loc] # Scale back the data to the original representation
    
    return inverted

# def invert_all_prediction_scaled(scaler, predictions, original_columns, horizons):
#     inverted = np.zeros(predictions.shape)
#     inverted2 = np.zeros(predictions.shape)
    
#     for col_idx, horizon in enumerate(horizons):
#         inverted[:,col_idx] = inverse_prediction_scale(
#             scaler, predictions[:,col_idx], 
#             original_columns,
#             "prediction_" + str(horizon))

#     for col_idx, horizon in enumerate(horizons):
#         inverted2[:,col_idx] = inverse_prediction_scale(
#             scaler, predictions[:,col_idx], 
#             original_columns,
#             "prediction_direction_" + str(horizon))
    
#     return inverted, inverted2

def invert_all_prediction_scaled(scaler, predictions, original_columns, horizons):
    inverted = np.zeros(predictions.shape)
    
    for col_idx, horizon in enumerate(horizons):
        inverted[:,col_idx] = inverse_prediction_scale(
            scaler, predictions[:,col_idx], 
            original_columns,
            "prediction_" + str(horizon))
    
    return inverted

def inverse_prediction_difference(predictions, original):
    return predictions + original

def invert_all_prediction_differences(predictions, original):
    inverted = predictions
    
    for col_idx, horizon in enumerate(horizons):
        inverted[:, col_idx] = inverse_prediction_difference(predictions[:,col_idx], original)
        
    return inverted

In [322]:
dataset = drop_duplicates(dataset)
dataset = impute_missing(dataset)
dataset.describe()

Number of duplicates: 189
Number of rows with nan: 18443


,AT,precipitation,humidity,pressure,wind_speed,wind_direction,nems4_AT,nems4_precipitation,nems4_humidity,nems4_pressure,nems4_wind_speed,nems4_wind_direction
count,75972.000000,75972.000000,75972.000000,75972.000000,75972.000000,75972.000000,75972.000000,75972.000000,75972.000000,75972.000000,75972.000000,75972.000000
mean,16.904299,0.076945,67.853630,1016.279315,4.111719,235.939425,16.397896,0.036491,70.878179,1016.174643,3.505294,210.255357
std,6.776936,1.069462,14.426722,7.002261,2.113328,107.526579,6.754892,0.279351,16.587193,7.140868,1.980608,107.176558
min,-3.500000,0.000000,8.000000,980.200000,0.000000,10.000000,-5.840000,0.000000,1.000000,982.000000,0.000000,0.000000
25%,11.800000,0.000000,59.000000,1012.500000,2.600000,160.000000,11.420000,0.000000,59.000000,1012.000000,2.050000,120.000000
50%,16.900000,0.000000,69.000000,1016.700000,4.100000,240.000000,16.600000,0.000000,72.000000,1017.000000,3.140000,212.000000
75%,22.500000,0.000000,78.000000,1020.400000,5.100000,340.000000,21.550000,0.000000,85.000000,1020.000000,4.620000,318.000000
max,35.300000,75.900000,100.000000,1041.600000,27.800000,360.000000,39.210000,16.000000,100.000000,1044.000000,22.320000,360.000000


In [323]:
#select features we're going to use
features = dataset[['wind_speed', 
                    'nems4_wind_speed',
#                     'wind_direction',
#                     'nems4_wind_direction', 
                    'AT', 
                    'nems4_AT', 
                    'humidity', 
                    'nems4_humidity',
                    'pressure',
                    'nems4_pressure']]

# features = dataset[['wind_speed', 
# #                     'wind_direction', 
#                     'AT', 
#                     'humidity', 
#                     'pressure']]

# the time horizons we're going to predict (in hours)
horizons = [1, 2, 4]

features = first_order_difference(features, features.columns)
features = derive_prediction_columns(features, 'wind_speed', horizons)
# features = derive_prediction_columns(features, 'wind_speed', 'wind_direction', horizons)

In [324]:
scaler = preprocessing.StandardScaler()
# scaler = preprocessing.MinMaxScaler()
scaled = scale_features(scaler, features)

scaled.describe()

,wind_speed,nems4_wind_speed,AT,nems4_AT,humidity,nems4_humidity,pressure,nems4_pressure,wind_speed_d,nems4_wind_speed_d,AT_d,nems4_AT_d,humidity_d,nems4_humidity_d,pressure_d,nems4_pressure_d,prediction_1,prediction_2,prediction_4
count,7.596700e+04,7.596700e+04,7.596700e+04,7.596700e+04,7.596700e+04,7.596700e+04,7.596700e+04,7.596700e+04,7.596700e+04,7.596700e+04,7.596700e+04,7.596700e+04,7.596700e+04,7.596700e+04,7.596700e+04,7.596700e+04,7.596700e+04,7.596700e+04,7.596700e+04
mean,-8.557838e-16,2.031495e-16,-4.652686e-17,-1.407402e-16,-3.981002e-18,-3.735390e-16,1.954055e-14,-3.668563e-15,6.373402e-18,1.331531e-17,-3.101645e-17,9.641214e-18,-9.549489e-19,-3.948850e-18,-4.204049e-16,2.120103e-16,-3.989482e-17,5.608331e-18,-2.210450e-18
std,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00
min,-1.945611e+00,-1.769794e+00,-3.010936e+00,-3.292083e+00,-4.148757e+00,-4.212724e+00,-5.152826e+00,-4.785885e+00,-1.445179e+01,-1.485796e+01,-9.609931e+00,-8.614736e+00,-7.817818e+00,-1.220231e+01,-3.845770e+01,-1.784291e+01,-1.445179e+01,-1.232098e+01,-9.370831e+00
25%,-7.153279e-01,-7.347696e-01,-7.532574e-01,-7.376861e-01,-6.136434e-01,-7.160871e-01,-5.396601e-01,-5.845459e-01,-7.190357e-01,-5.231252e-01,-5.709047e-01,-5.341246e-01,-5.211921e-01,-5.229409e-01,-3.510126e-01,1.264732e-04,-7.190316e-01,-5.575243e-01,-6.507894e-01
50%,-5.549267e-03,-1.844395e-01,-6.977215e-04,2.989137e-02,7.951622e-02,6.764186e-02,6.019424e-02,1.156774e-01,-4.353685e-05,-4.688765e-05,-9.516643e-02,-1.550767e-01,-4.573807e-06,1.606267e-05,2.011109e-04,1.264732e-04,-3.880462e-05,-1.467768e-05,-3.997565e-05
75%,4.676365e-01,5.627977e-01,8.256423e-01,7.626799e-01,7.033598e-01,8.513708e-01,5.886374e-01,5.358113e-01,4.313518e-01,4.607602e-01,3.805718e-01,3.704214e-01,5.211830e-01,5.229730e-01,3.514148e-01,1.264732e-04,4.313569e-01,5.574949e-01,6.507094e-01
max,1.120895e+01,9.499351e+00,2.714420e+00,3.377033e+00,2.228311e+00,1.755673e+00,3.616474e+00,3.896883e+00,1.926895e+01,1.295237e+01,9.705041e+00,8.554409e+00,8.165267e+00,1.133075e+01,2.616562e+01,1.784316e+01,1.926897e+01,1.321296e+01,1.093255e+01


In [325]:
def prepare_test_train(data, features, predictions, sequence_length, split_percent=0.9):
    
    num_features = len(features)
    num_predictions = len(predictions)
    
    # make sure prediction cols are at end
    columns = features + predictions
    
    data = data[columns].values
    
    print("Using {} features to predict {} horizons".format(num_features, num_predictions))
    
    result = []
    for index in range(len(data) - sequence_length+1):
        result.append(data[index:index + sequence_length])

    result = np.array(result)
    # shape (n_samples, sequence_length, num_features + num_predictions)
    print("Shape of data: {}".format(np.shape(result)))
    
    row = round(split_percent * result.shape[0])
    train = result[:row, :]
    print("Shape of train: {}".format(np.shape(train)))
    
    X_train = train[:, :, :-num_predictions]
    y_train = train[:, -1, -num_predictions:]
    X_test = result[row:, :, :-num_predictions]
    y_test = result[row:, -1, -num_predictions:]
    
    print("Shape of X train: {}".format(np.shape(X_train)))
    print("Shape of y train: {}".format(np.shape(y_train)))
    print("Shape of X test: {}".format(np.shape(X_test)))
    
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], num_features))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], num_features))
    
    y_train = np.reshape(y_train, (y_train.shape[0], num_predictions))
    y_test = np.reshape(y_test, (y_test.shape[0], num_predictions))
    
    return X_train, y_train, X_test, y_test, row

In [326]:
sequence_length = 72

prediction_cols = ['prediction_' + str(h) for h in horizons]
# prediction_cols_2 = ['prediction_direction_' + str(h) for h in horizons]
# prediction_cols = prediction_cols + prediction_cols_2

feature_cols = ['wind_speed', 'nems4_wind_speed', 
#                 'wind_direction_d', 'nems4_wind_direction_d',
                'AT', 'nems4_AT', 
                'humidity', 'nems4_humidity', 
                'pressure', 'nems4_pressure']

# feature_cols = ['wind_speed_d', 'nems4_wind_speed_d', 
# #                 'wind_direction_d', 'nems4_wind_direction_d',
#                 'AT_d', 'nems4_AT_d', 
#                 'humidity_d', 'nems4_humidity_d', 
#                 'pressure_d', 'nems4_pressure_d']

# feature_cols = ['wind_speed_d', 
# #                 'wind_direction_d', 
#                 'AT_d', 
#                 'humidity_d', 
#                 'pressure_d']

X_train, y_train, X_test, y_test, row_split = prepare_test_train(
    scaled,
    feature_cols,
    prediction_cols,
    sequence_length,
    split_percent = 0.8)

Using 8 features to predict 3 horizons
Shape of data: (75896, 72, 11)
Shape of train: (60717, 72, 11)
Shape of X train: (60717, 72, 8)
Shape of y train: (60717, 3)
Shape of X test: (15179, 72, 8)


In [327]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

#(-1 is because we only take the last y row in each sequence)
sequence_offset = sequence_length - 1

# validate train
inverse_scale = invert_all_prediction_scaled(scaler, y_train, scaled.columns, horizons)

assert(mean_squared_error(
    features[prediction_cols][sequence_offset:row_split+sequence_offset], 
    inverse_scale) < 1e-10)


undiff_prediction = invert_all_prediction_differences(
    inverse_scale, 
    features['wind_speed'][sequence_offset:row_split+sequence_offset])

for i, horizon in enumerate(horizons):
    assert(mean_squared_error(
        features['wind_speed'][sequence_offset+horizon:row_split+sequence_offset+horizon], 
        undiff_prediction[:,i]) < 1e-10)

    
# validate test
inverse_scale = invert_all_prediction_scaled(scaler, y_test, scaled.columns, horizons)

assert(mean_squared_error(
    features[prediction_cols][sequence_offset+row_split:], 
    inverse_scale) < 1e-10)

undiff_prediction = invert_all_prediction_differences(
    inverse_scale, 
    features['wind_speed'][sequence_offset+row_split:])

for i, horizon in enumerate(horizons):
    assert(mean_squared_error(
        features['wind_speed'][sequence_offset+row_split+horizon:], 
        undiff_prediction[:-horizon,i]) < 1e-10)

In [328]:
# ### Build the LSTM Model

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import LSTM
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import RMSprop, Adam


def build_model(layers):
    model = Sequential()
    
#     model.add(Conv1D(layers[1], 12, input_shape=(None, layers[0])))

    model.add(LSTM(
            layers[1],
            input_shape=(None, layers[0]),
            return_sequences=True))
    model.add(Dropout(0.3))
    
#     model.add(LSTM(layers[3], return_sequences=True))
#     model.add(Dropout(0.2))
    
    model.add(LSTM(layers[5]))
    model.add(Dropout(0.3))
    
    model.add(Dense(layers[4]))
    model.add(Activation('linear'))
    
    model.compile(loss="mean_squared_error", optimizer='rmsprop', metrics=['accuracy'])
    
    print(model.summary())
          
    return model

In [ ]:
from keras.callbacks import TensorBoard, ReduceLROnPlateau

def run_network(X_train, y_train, X_test, y_test, layers, epochs, batch_size=16):
    model = build_model(layers)
    history = None
    
    try:
        history = model.fit(
            X_train, y_train, 
            batch_size=batch_size, 
            epochs=epochs,
            validation_split=0.1,
            callbacks=[
                TensorBoard(log_dir='./tensorboard', write_graph=True),
                ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=15, 
                                verbose=1, mode='auto', min_lr=0.0001),
                EarlyStopping(monitor='val_loss', patience=30, verbose=1, mode='auto'),
                ModelCheckpoint('./model/best.hdf5', monitor='val_loss', verbose=1, 
                                save_best_only=True, mode='auto')
            ])
    except KeyboardInterrupt:
        print("\nTraining interrupted")
    
    predicted = model.predict(X_test, batch_size=batch_size)
    scores = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
    
    return model, predicted, history, scores

In [ ]:
model, predicted, history, scores = run_network(
    X_train, 
    y_train, 
    X_test,
    y_test,
    layers=[X_train.shape[2], 128, 30, 20, y_train.shape[1], 32],
    epochs=200)

print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_133 (LSTM)              (None, None, 128)         70144     
_________________________________________________________________
dropout_133 (Dropout)        (None, None, 128)         0         
_________________________________________________________________
lstm_134 (LSTM)              (None, 32)                20608     
_________________________________________________________________
dropout_134 (Dropout)        (None, 32)                0         
_________________________________________________________________
dense_66 (Dense)             (None, 3)                 99        
_________________________________________________________________
activation_66 (Activation)   (None, 3)                 0         
Total params: 90,851
Trainable params: 90,851
Non-trainable params: 0
_________________________________________________________________
None
T

In [ ]:
import matplotlib.pyplot as plt
# loss
fig = plt.figure(figsize=(12, 5))
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.savefig('./image/loss.png')
plt.show()

# accuracy
fig = plt.figure(figsize=(12, 5))
plt.plot(history.history['acc'], label='train_acc')
plt.plot(history.history['val_acc'], label='val_acc')
plt.legend()
plt.savefig('./image/acc.png')
plt.show()


# print("*********************************************************")
# print(type(predicted)) # <class 'numpy.ndarray'>
# print(predicted.shape) # (7594, 6)
# print("*********************************************************")
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

print("MAE {:.3}, RMSE {:.3f}".format(
    mean_absolute_error(y_test, predicted),
    sqrt(mean_squared_error(y_test, predicted))))

for i, horizon in enumerate(horizons):
    print("wind speed: MAE {:.3f}, RMSE {:.3f} for horizon {}".format(
        mean_absolute_error(y_test[:,i], predicted[:,i]),
        sqrt(mean_squared_error(y_test[:,i], predicted[:,i])),
        horizon))

# for i, horizon in enumerate(horizons):
#     print("wind direction: MAE {:.3f}, RMSE {:.3f} for horizon {}".format(
#         mean_absolute_error(y_test[:,i+3], predicted[:,i+3]),
#         sqrt(mean_squared_error(y_test[:,i+3], predicted[:,i+3])),
#         horizon))

In [ ]:
# inverse_scale, inverse_scale2 = invert_all_prediction_scaled(scaler, predicted, scaled.columns, horizons)
inverse_scale = invert_all_prediction_scaled(scaler, predicted, scaled.columns, horizons)
# print(inverse_scale.shape) # (7594, 6)
# print("*********************************************************************")
# print(inverse_scale2.shape) # (7594, 6)

predicted_signal = invert_all_prediction_differences(
    inverse_scale, 
    features['wind_speed'][sequence_offset+row_split:])
# print(predicted_signal.shape) # (7594, 6)
print(predicted_signal[:5])
# print("************************************************************************")

# predicted_signal2 = invert_all_prediction_differences(
#     inverse_scale2, 
#     features['wind_direction'][sequence_offset+row_split:])
# print(predicted_signal2[:5])
# print("***************************************************************************")

for i, horizon in enumerate(horizons):
    a = features['wind_speed'][sequence_offset+row_split+horizon:]
    p = predicted_signal[:-horizon, i]
    
    print("Real scale wind speed: MAE {:.3f}, RMSE {:.3f} for horizon {}".format(
            mean_absolute_error(a, p),
            sqrt(mean_squared_error(a, p)),
            horizon))
    
# for i, horizon in enumerate(horizons):
#     a2 = features['wind_direction'][sequence_offset+row_split+horizon:]
#     p2 = predicted_signal2[:-horizon, i]

#     print("Real scale wind direction: MAE {:.3f}, RMSE {:.3f} for horizon {}".format(
#             mean_absolute_error(a2, p2),
#             sqrt(mean_squared_error(a2, p2)),
#             horizon)) 


In [ ]:
from time import time
def SLSTM(epoch=2, batch_size=1024, layers=[35, 20]):
    model2 = Sequential()
    
    model2.add(LSTM(layers[0], input_shape=(None, X_train.shape[2]), return_sequences=True))
    model2.add(Dropout(0.2))
    
    model2.add(LSTM(layers[1]))
    model2.add(Dropout(0.2))
               
    model2.add(Dense(y_train.shape[1]))
    model.add(Activation('linear'))

    model2.compile(loss='mae', optimizer='rmsprop')
    t0 = time()
    model2.fit(X_train, y_train, epochs=epoch, batch_size=batch_size, verbose=0)

    predicted = model2.predict(X_test, batch_size=batch_size)
    inverse_scale = invert_all_prediction_scaled(scaler, predicted, scaled.columns, horizons)
    predicted_signal = invert_all_prediction_differences(
        inverse_scale, 
        features['wind_speed'][sequence_offset+row_split:])
    for i, horizon in enumerate(horizons):
        a = features['wind_speed'][sequence_offset+row_split+horizon:]
        p = predicted_signal[:-horizon, i]
        MAE = mean_absolute_error(a, p)
        print("Real scale wind speed: MAE {:.3f}, RMSE {:.3f} for horizon {}".format(
        mean_absolute_error(a, p),
        sqrt(mean_squared_error(a, p)),
        horizon))

    deltatime = time()-t0
    print("MAE=%.5f, 消耗时间=%.4f 秒" % (MAE, deltatime))

# for batch_size in [128, 256, 512, 1024]:
#     SLSTM(batch_size=batch_size)
# print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
for layers in [[128, 64], [64, 64], [64, 32], [32, 32]]:
    SLSTM(layers=layers)